In [1]:
### (차원축소 이어서)
### 지난과정 복습 
## 차원축소를 해야 하는 이유(목적)
## 1. 데이터 시각화 가능 -> 데이터의 패턴 이해 용이
## 2. 노이즈 제거 
## 3. 메모리 절약 -> 모델 성능 개선

## 차원 축소 method: 
## 1.주성분분석(PCA) 
##   특성들 중에서 클래스를 분류하는데 두드러지게 중요한 특성을 
##   추출함으로써 차원을 축소 
## 데이터 집합에 대한 공분산 행렬 구함 
## -> 공분산 행렬로부터 고유값과 고유벡터 계산
## -> 고유값 목록에서 값이 높은 k개의 인덱스를 구함 
## PCA는 데이터 압축, 얼굴 인식 기능등에 활용됨 

## 2.LDA(선형 판별 분석): 클래스를 최대한 분리하는 성분을 축으로 차원 축소 

## 3.NMF(비음수 행렬 인수분해)
##   비지도 학습 알고리즘
##   음수가 아닌 특성과 계수 값을 찾음 
##   주성분 계수가 양수(0보다 크거나 같아야 함)
##   음수가 아닌 특성들에 대해서 가중치의 합으로 데이터를 분해 
##   여러 데이터 특성 중에서 주요 독립 특성을 추출하는 방법으로 차원을 축소 

## 4.SVD(특이값 분해)를 이용한 차원축소 
##   행렬을 대각화 하는 방법으로 정방행렬이 아니어도 mXn행렬에대해 적용 가능 
##   직교하는 벡터 집합에 대해서 선형 변환 후에 그 크기는 변하지만 여전히 직교할 수 
##   있게되는 직교집합을 찾음으로서 차원을 축소하는 방식 
## 데이터 압축, 유사도 분석등에 많이 활용됨 
## TSVD는 희소 특성 행렬 

In [ ]:
### 특성 선택을 이용한 차원 축소
### dimension reduction2(특성 선택을 사용한 차원축소).pdf 참고 

'''
특성 선택
 특성 선택 (feature selection) : 고품질의 정보가 많은 특성은 선택하고 
                                  덜 유용한 특성은 버리는 방식
 필터(filter) - 통계적인 속성을 조사하여 가장 뛰어난 특성을 선택
 래퍼(wrapper) - 시행착오를 통해 가장 높은 품질의 예측을 만드는 특성의 부분 조합을 
                  찾아 선택
 임베디드(embedded) - 학습 알고리즘의 훈련 단계를 확장하거나 일부로 구성하여 
                       가장 좋은 특성의 부분 조합을 선택
'''

In [7]:
'''
분산을 기준으로 수치 특성 선택
 분산 기준 설정(variance thresholding)은 가장 기본적인 특성 선택 방법 중 하나입니다.
 분산이 높은 특성보다 분산이 낮은 특성이 효과적이거나 유용하지 않다는 아이디어에 기반합니다.
 분산 기준 설정 VT(VarianceThreshold)는 특성의 제곱 단위이므로 특성의 단위가 서로 다르면 VT가 동작하지 않습
니다.
 분산의 기준값을 수동으로 선택하기 때문에 어떤 값이 좋은지 판단할 수 있어야 합니다.
'''
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

iris = datasets.load_iris() #데이터를 로드

features = iris.data # 특성과 타깃을 만듭니다
target = iris.target

thresholder = VarianceThreshold(threshold=.5) # 기준값을 만듭니다.

# 기준값보다 높은 특성을 선택합니다.
features_high_variance = thresholder.fit_transform(features) 

features_high_variance[0:3] # 선택한 특성을 확인

array([[5.1, 1.4, 0.2],
       [4.9, 1.4, 0.2],
       [4.7, 1.3, 0.2]])

In [4]:
thresholder.variances_ # 분산 확인

array([0.68112222, 0.18871289, 3.09550267, 0.57713289])

In [5]:
'''
분산을 기준으로 수치 특성 선택(이어서)
 특성이 평균이 0이고 단위 분산으로 표준화 되어 있으면 
  분산 기준 선택 방식은 올바르게 동작하지 않습니다.
'''
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() # 특성 행렬을 표준화합니다.
features_std = scaler.fit_transform(features)

selector = VarianceThreshold() # 각 특성의 분산을 계산합니다.
selector.fit(features_std).variances_

array([1., 1., 1., 1.])

In [8]:
'''
분산을 기준으로 이진 특성 선택
 이진 범주형 특성에서 베르누이 확률 변수의 분산이 기준값 이상인 특성을 선택할 수 있도록 
  분산이 낮은 특성(적은 정보를 가진 특성)을 삭제합니다
 p는 클래스 1의 샘플 비율입니다.
 p값을 설정하여 샘플의 대다수가 한 개의 클래스에 속한 특성을 삭제할 수 있습니다.
 VarainceThreshold 클래스는 수치 특성, 이진 특성에 상관없이 
  넘파이 var 함수를 사용하여 분산을 계산합니다.
'''
from sklearn.feature_selection import VarianceThreshold

features = [[0, 1, 0], # 예제 특성 행렬
            [0, 1, 1], # 특성 0: 80%가 클래스 0
            [0, 1, 0], # 특성 1: 80%가 클래스 1
            [0, 1, 1], # 특성 2: 60%가 클래스 0, 40%는 클래스 1
            [1, 0, 0]]

# 분산을 기준으로 선택합니다.
thresholder = VarianceThreshold(threshold=(.75 * (1 - .75)))
thresholder.fit_transform(features)
thresholder.variances_


array([0.16, 0.16, 0.24])

In [9]:
import numpy as np
np.var(features, axis=0) #넘파이 var 함수를 사용하여 분산을 계산합니다

array([0.16, 0.16, 0.24])

In [ ]:
'''
분산을 기준으로 이진 특성 선택(이어서)
 이진 특성에 var 함수를 사용하는 것은 이진 특성일 때 베르누이 확률 변수의 분산과 
  같기 때문입니다.
 이진 특성의 평균 μ는 클래스 1의 샘플 비율과 같습니다.
 var 함수로 이진 특성의 분산을 계산하면 베르누이 확률 변수의 분산 p(1-p)와 같습니다.
   (결과가 두 개인 시행의 결과에 대하여 성공을 , 실패를 으로 표시하는 확률변수)
 threshold 매개변수의 기본값은 0으로 모든 특성을 선택합니다.
'''

In [10]:
'''
상관관계가 큰 특성 삭제
 두 가지 특성의 상관관계가 크다면, 담고 있는 정보가 매우 비슷하므로 
  중복된 특성을 포함하는 것과 같습니다.
 특성 행렬에서 상관관계 행렬을 사용하여 상관관계가 큰 특성을 확인하고 
  이들 중 하나를 삭제합니다.
1. 모든 특성에 대한 상관관계 행렬을 만듭니다.
2. 상관관계 행렬의 상삼각 행렬(upper triangle matrix)을 살펴서 크게 상관된 특성의 쌍을 확인합니다
3. 특성 행렬에서 이런 특성 중 하나를 삭제합니다.
'''

import pandas as pd
import numpy as np
# 상관관계가 큰 두 개의 특성을 가진 특성 행렬을 만듭니다.
features = np.array([[1, 1, 1], [2, 2, 0], [3, 3, 1], [4, 4, 0], [5, 5, 1],
                     [6, 6, 0], [7, 7, 1], [8, 7, 0], [9, 7, 1]])

dataframe = pd.DataFrame(features) # 특성 행렬을 DataFrame으로 변환
corr_matrix = dataframe.corr().abs() # 상관관계 행렬을 만듭니다.

# 상관관계 행렬의 상삼각(upper triangle) 행렬을 선택합니다.
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# 상관 계수가 0.95보다 큰 특성 열의 인덱스를 찾습니다.
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [11]:
'''
상관관계가 큰 특성 삭제(이어서)
 특성 행렬에서 상관관계 행렬을 사용하여 상관관계가 큰 특성을 확인하고 
  이들 중 하나를 삭제합니다.
'''
dataframe.drop(dataframe.columns[to_drop], axis=1).head(3) # 특성을 삭제합니다.

dataframe.corr() #상관관계 행렬

,0,1,2
0,1.000000,0.976103,0.000000
1,0.976103,1.000000,-0.034503
2,0.000000,-0.034503,1.000000


In [12]:
upper #상관관계 행렬의 상삼각 행렬

,0,1,2
0,NaN,0.976103,0.000000
1,NaN,NaN,0.034503
2,NaN,NaN,NaN


In [13]:
#상관관계 행렬은 넘파이 corrcoef()로 구할 수 있습니다.
#corrcoef()는 특성이 행에 놓여 있을 것으로 가정합니다.
#특성이 열에 놓여 있다고 알려주려면 rowvar 매개변수를 False로 지정합니다.
np.corrcoef(features, rowvar=False)

array([[ 1.        ,  0.97610336,  0.        ],
       [ 0.97610336,  1.        , -0.03450328],
       [ 0.        , -0.03450328,  1.        ]])

In [14]:
# np.triu()는 주어진 배열에서 상삼각 행렬을 추출하여 반환합니다.
# 매개변수 k가 기본값 0이면 반환되는 행렬에 대각원소가 포함됩니다.
# k값이 커질수록 대각원소에서 k만큼 떨어진 삼각행렬을 반환합니다.
# 예) k=2일 경우 주대각선에서 2만큼 떨어진 원소부터 포함됩니다.
np.triu(np.ones((4, 4)), k=2)

array([[0., 0., 1., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [15]:
# np.tril()는 주어진 배열에서 하삼각 행렬을 추출 반환합니다.
np.tril(np.ones((4, 4)), k=0) #대각원소 포함 

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [16]:
'''
분류 작업에 관련 없는 특성 삭제
 범주형 타깃 벡터에서 관련 없는 특성을 삭제하려면 각 특성과 타깃 벡터 사이의 
  카이제곱 통계를 계산합니다.
 카이제곱 통계는 두 범주형 벡터의 독립성을 평가합니다.
 카이제곱 통계는 범주형 특성의 각 클래스별 샘플 빈도와 
  이 특성이 타깃 벡터와 독립적이라면 기대할 수 있는 값사이의 차이입니다
 카이제곱 특성은 관찰 빈도와 전혀 관계가 없다고 기대하는 빈도 사이에 
  얼마나 큰 차이가 있는지 알려주는 하나 의 숫자입니다
 특성과 타깃 벡터 사이의 카이제곱 통계를 계산하면 둘 사이의 독립성을 측정할 수 있습니다.
'''
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

iris = load_iris() # 데이터 로드
features = iris.data
target = iris.target
features = features.astype(int) # 범주형 데이터를 정수형으로 변환

chi2_selector = SelectKBest(chi2, k=2) # 카이제곱 통계값이 가장 큰 특성 두 개를 선택
features_kbest = chi2_selector.fit_transform(features, target)

print("원본 특성 개수:", features.shape[1]) # 결과 확인
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 2


In [17]:
'''
분류 작업에 관련 없는 특성 삭제(이어서)
 특성 선택에서 카이제곱을 사용하려면 각 특성과 타깃 벡터 사이의 카이제곱 통계를 계산하고 카이제곱 통계가 가
장 좋은 특성을 선택해야 합니다.
 사이킷런에서는 SelectBest를 사용하여 통곗값이 가장 좋은 특성을 선택할 수 있습니다.
 매개변수 k는 선택하려는 특성의 개수를 결정합니다
 카이제곱 통계는 두 범주형 벡터 사이에서만 계산할 수 있습니다.
 특성 선택으로 카이제곱을 사용하려면 타깃 벡터와 특성이 범주형이어야 합니다.
 수치형 특성이 있다면 수치형을 범주형 특성으로 변환하여 카이제곱 특성을 사용할 수 있습니다.
 카이제곱 방식을 사용하려면 모든 값이 음수가 아니어야 합니다.
'''
'''
 특성이 수치형 특성이라면 f_classif 사용하여 각 특성과 타깃 벡터 사이에 
  분산 분석(ANOVA)와 F-값 통계를 계산 할 수 있습니다.
 F-값 점수는 타깃 벡터로 수치형 특성을 그룹핑하여 각 그룹의 평균이 크게 차이 나는지 
  평가합니다.
 예] 이진 타깃 벡터인 성별과 수치형 특성인 시험 점수가 있다면, 
       F-값 점수는 남성의 평균 테스트 점수가 
       여성의 평균 테스트 점수보다 다른지를 설명합니다.
'''

# F-값이 가장 높은 특성 두 개를 선택합니다.
fvalue_selector = SelectKBest(f_classif, k=2)
features_kbest = fvalue_selector.fit_transform(features, target)

print("원본 특성 개수:", features.shape[1]) # 결과 확인
print("줄어든 특성 개수:", features_kbest.shape[1])

# 특정 특성 개수를 선택하는 대신 Selectpercentile를 사용하여 
# 특성의 상위 n 퍼센트를 선택할 수 있습니다.
from sklearn.feature_selection import SelectPercentile

# 가장 큰 F-값의 상위 75% 특성을 선택합니다.
fvalue_selector = SelectPercentile(f_classif, percentile=75)
features_kbest = fvalue_selector.fit_transform(features, target)
print("원본 특성 개수:", features.shape[1]) # 결과 선택
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 2
원본 특성 개수: 4
줄어든 특성 개수: 3


In [18]:
'''
분류 작업에 관련 없는 특성 삭제(이어서)
 카이제곱 계산
'''
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [19]:
#특성 행렬의 차원을 (3, 50, 4)로 바꾸어 클래스별 합을 구합니다.
observed = np.sum(features.reshape(3, 50, 4), axis=1)
observed

array([[230, 152,  50,   0],
       [274, 116, 191,  50],
       [304, 129, 255,  79]])

In [21]:
#특성 타깃과 전혀 관계없다면 기대 빈도는 전체 합을 클래스 개수 3으로 나눈 값이 됩니다.
expected = features.sum(axis=0) / 3
expected

array([269.33333333, 132.33333333, 165.33333333,  43.        ])

In [22]:
#카이제곱 공식에 위헤서 구한 observed와 expected를 대입합니다.
np.sum((observed - expected)**2 / expected, axis=0)

array([ 10.28712871,   5.02267003, 133.06854839,  74.27906977])

In [23]:
#카이제곱 값이 큰 세 번째, 네 번째 특성이 선택됩니다. 
#chi2_selector객체의 scores_속성에 저장
chi2_selector.scores_

array([ 10.28712871,   5.02267003, 133.06854839,  74.27906977])

In [30]:
'''
분류 작업에 관련 없는 특성 삭제(이어서)
 ANOVA는 각 특성이 독립적으로 평가되기 때문에 일변량 분석이라고도 부릅니다.
'''
#ANOVA 직접 계산
##전체 평균과 클래스 평균을 계산
total_mean = np.mean(features, axis=0)
total_mean

array([5.38666667, 2.64666667, 3.30666667, 0.86      ])

In [31]:
class_mean = np.mean(features.reshape(3, 50, 4), axis=1)
class_mean

array([[4.6 , 3.04, 1.  , 0.  ],
       [5.48, 2.32, 3.82, 1.  ],
       [6.08, 2.58, 5.1 , 1.58]])

In [26]:
#ss_total 계산
ss_between = np.sum(50 * (class_mean - total_mean)**2, axis=0)
ss_between

array([ 55.41333333,  13.29333333, 440.01333333,  63.88      ])

In [27]:
ss_total = np.sum((features - total_mean)**2, axis=0)
ss_total

array([105.57333333,  42.27333333, 467.89333333,  76.06      ])

In [28]:
#ss_beteen과 ss_tatal을 F-값 공식에 대입
f = (ss_between/(3-1)) / ((ss_total-ss_between)/(150-3))
f

array([  81.19776715,   33.71497585, 1160.00645624,  385.48275862])

In [29]:
fvalue_selector.scores_ #F-값 scores_속성에서 확인

array([  81.19715 ,   33.715004, 1160.0116  ,  385.483   ], dtype=float32)

In [32]:
'''
재귀적 특성 제거 (recursive feature elimination)
 모델 성능이 나빠질 때까지 특성을 제거하면서 반복적으로 모델을 훈련하면
  자동으로 최선의 특성만 남게 됩니다.
 사이킷런의 RFECV를 사용하여 재귀적 특성 제거를 교차 검증(cross-validation)으로 
  수행할 수 있습니다
'''
'''
재귀적 특성제거는
 교차검증(CV)를 사용하여 RFE 과정에서 남길 특성의 최적 개수를 찾을 수 있습니다.
 구체적으로 매 반복 후에 CV 를 사용한 RFE에서 교차검증을 사용하여 모델을 평가합니다.
 특성을 제거한 후에 모델의 CV 결과가 향상되었다면 다음 반복으로 계속 진행합니다.
 어떤 특성을 제거한 후에 모델의 CV 결과가 더 나빠지면 삭제한 특성을 다시 복원하고 이 특성 조합을 최선으로
선택합니다.
 사이킷런의 RFECV는 CV를 사용한 REF 구현으로 estimator 매개변수에는 훈련한 모델의 객체를 전달합니다
 step 매개변수는 매 반복에서 삭제할 특성의 개수나 비율을 정합니다.
 scoring 매개변수에는 교차검증 동안 사용할 모델의 평가 지표를 설정합니다.
'''
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn import datasets, linear_model

# 특성 행렬과 타깃 벡터를 생성합니다.
features, target = make_regression(n_samples = 10000,
                                   n_features = 100,
                                   n_informative = 2,
                                   random_state = 1)

# 선형 회귀 모델을 만듭니다.
ols = linear_model.LinearRegression()

# 재귀적으로 특성을 제거합니다.
rfecv = RFECV(estimator=ols, step=1, scoring="neg_mean_squared_error")
rfecv.fit(features, target)
rfecv.transform(features)

array([[ 0.00850799,  0.7031277 , -0.925066  ],
       [-1.07500204,  2.56148527,  0.4746258 ],
       [ 1.37940721, -1.77039484, -0.39616889],
       ...,
       [-0.80331656, -1.60648007,  0.25068305],
       [ 0.39508844, -1.34564911, -1.35054293],
       [-0.55383035,  0.82880112,  0.14050409]])

In [33]:
rfecv.n_features_ # 최선의 특성 개수

3

In [34]:
rfecv.support_ # 선택된 특성이 표시된 불리언 마스크

array([False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [35]:
rfecv.ranking_ # 특성의 순위: 최고(1)에서 최악(96)까지

array([69, 15, 94, 45, 13,  1, 35,  6, 31, 81, 36, 25,  2,  7, 53, 40, 27,
       46, 75, 65, 61, 68, 92,  9, 39, 48, 98, 96, 47, 71, 19, 37, 11, 20,
       50,  4, 33, 42, 67,  1, 43, 63, 85, 86, 56, 60,  5, 16,  8, 55, 93,
       73, 10, 76,  1, 77, 52, 24, 58, 62, 21, 82, 72, 90, 80, 91, 18, 30,
       57, 89, 64, 51, 59, 17, 28, 32, 49, 66, 87, 84, 38, 88, 34, 44, 14,
       79, 41, 12, 29, 23,  3, 78, 22, 95, 26, 70, 54, 83, 74, 97])

In [36]:
from sklearn.feature_selection import RFE

rfe = RFE(estimator=ols, n_features_to_select=3)


In [37]:
rfe.fit(features, target)


RFE(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                               normalize=False),
    n_features_to_select=3, step=1, verbose=0)

In [38]:
rfe.transform(features)


array([[ 0.00850799,  0.7031277 , -0.925066  ],
       [-1.07500204,  2.56148527,  0.4746258 ],
       [ 1.37940721, -1.77039484, -0.39616889],
       ...,
       [-0.80331656, -1.60648007,  0.25068305],
       [ 0.39508844, -1.34564911, -1.35054293],
       [-0.55383035,  0.82880112,  0.14050409]])

In [39]:
# rfe객체가 선택한 특성이 rfecv 객체가 선택한 특성과 동일한지 확인하기 위해 
# 불리언 마스크를 비교
np.all(rfe.support_ == rfecv.support_)

True

In [ ]:
'''
재귀적 특성 제거 (recursive feature elimination)(이어서)
 steop 매개변수의 기본값은 1입니다
 scoring 매개변수를 지정하지 않으면 estimator에 지정된 모델의 score ()fmf tkdydgkqselk.
 n_jobs 매개변수에서 교차검증을 위해 사용할 CPU 코어 수를 지정할 수 있습니다
 n_jobs 매개변수의 기본값은 1입니다.
 cv 매개변수는 k-폴드 교차검증의 k값을 결정합니다. 기본값은 5입니다.
 사이킷런은 교차검증을 사용하지 않는 재귀적 특성 제거 방법인 RFE 클래스를 제공합니다.
 RFECV 클래스와 마찬가지로 남길 최소 특성의 개수를 n_features_to_select 매개변수에서 지정할 수 있지만 RFECV
와 달리 입력 특성의 절반이 기본값입니다.
'''